<div style="text-align: right" align="right"><i>Peter Norvig, 1–25 Dec 2021</i></div>

# Advent of Code 2021

I'm going to solve some [Advent of Code 2021](https://adventofcode.com/) (AoC) puzzles, but I won't be competing for time. 

I also won't explain each puzzle here; you'll have to click on each day's link (e.g. [Day 1](https://adventofcode.com/2021/day/1)) to understand the puzzle. 

Part of the idea of AoC is that you have to make some design choices to solve part 1 before you get to see the description of part 2. So there is a tension of wanting the solution to part 1 to provide general components that might be re-used in part 2, without falling victim to [YAGNI](https://en.wikipedia.org/wiki/You_aren%27t_gonna_need_it). Except for errors, I will show the code as I developed it; I won't go back and refactor the code for part 1 when I see part 2.

# Day 0: Imports and Utilities

Below are the imports and utility functions that I found to be generally useful for these kinds of problems. But first two functions that I will use each day, `parse` and `answer`.  I will start by writing something like this for part 1 of Day 1:

    in1 = parse(1, int)
    def solve_it(numbers): return ...
    solve(in1)
    
That is, `parse(1, int)` will parse the data file for day 1 in the format of one integer per line; then some function (here `solve_it`) will (hopefully) compute the correct answer. I'll then submit the answer to AoC and verify it is correct. If it is, I'll move on to solve part 2. When I get them both done, I'll use the function `answers` to assert the correct answers. So it will look like this:

    in1 = parse(1, int)
    
    def solve_it(numbers): return ...
    answer(1.1, solve_it(in1), ...)
    
    def solve_it2(numbers): return ...
    answer(1.2, solve_it2(in1), ...)
    


In [1]:
def parse(day, parser=str, sep='\n') -> tuple:
    "Split the day's input file into sections separated by `sep`, and apply `parser` to each."
    sections = open(f'AOC2021/input{day}.txt').read().rstrip().split(sep)
    return mapt(parser, sections)

def answer(puzzle_number, got, expected) -> bool:
    """Verify the answer we got was expected."""
    assert got == expected, f'For {puzzle_number}, expected {expected} but got {got}.'
    return True

In [2]:
from __future__  import annotations
from collections import Counter, defaultdict, namedtuple, deque
from itertools   import permutations, combinations, chain, count as count_from, product as cross_product
from functools   import lru_cache
from typing      import Dict, Tuple, Set, List, Iterator, Optional, Union

import operator
import math
import ast
import sys
import re

Char    = str # Type used to indicate a single character
cat     = ''.join
flatten = chain.from_iterable
cache   = lru_cache(None)

In [3]:
def quantify(iterable, pred=bool) -> int:
    "Count the number of items in iterable for which pred is true."
    return sum(1 for item in iterable if pred(item))

def first(iterable, default=None) -> object:
    "Return first item in iterable, or default."
    return next(iter(iterable), default)

def rest(sequence) -> object: return sequence[1:]

def multimap(items: Iterable[Tuple]) -> dict:
    "Given (key, val) pairs, return {key: [val, ....], ...}."
    result = defaultdict(list)
    for (key, val) in items:
        result[key].append(val)
    return result

def prod(numbers) -> float: # Will be math.prod in Python 3.8, but I'm in 3.7
    "The product of an iterable of numbers." 
    result = 1
    for n in numbers:
        result *= n
    return result

def ints(text: str) -> Tuple[int]:
    "Return a tuple of all the integers in text, ignoring non-numbers."
    return tuple(map(int, re.findall('-?[0-9]+', text)))

def sign(x) -> int:
    """The sign of a number: +1, 0, or -1."""
    return (0 if x == 0 else +1 if x > 0 else -1)

Atom = Union[float, int, str]

def atoms(text: str, sep=None) -> Tuple[Atom]:
    "Parse text into atoms (numbers or strs)."
    return tuple(map(atom, text.split(sep)))

def atom(text: str) -> Atom:
    "Parse text into a single float or int or str."
    try:
        val = float(text)
        return round(val) if round(val) == val else val
    except ValueError:
        return text
    
def dotproduct(A, B) -> float: return sum(a * b for a, b in zip(A, B))

def mapt(fn, *args):
    "map(fn, *args) and return the result as a tuple."
    return tuple(map(fn, *args))

# [Day 1](https://adventofcode.com/2021/day/1): Sonar Sweep

The input is a list of integers.

1. How many numbers increase from the previous number?
2. How many sliding windows of 3 numbers increase from the previous window?

In [4]:
in1 = parse(1, int)

In [5]:
def increases(nums: List[int]) -> int:
    """How many numbers are bigger than the previous one?"""
    return quantify(nums[i] > nums[i - 1] 
                    for i in range(1, len(nums)))

answer(1.1, increases(in1), 1400)

True

In [6]:
def window_increases(nums: List[int], w=3) -> int:
    """How many sliding windows of w numbers have a sum bigger than the previous window?"""
    return quantify(sum(nums[i:i+w]) > sum(nums[i-1:i-1+w])
                    for i in range(1, len(nums) + 1 - w))

answer(1.2, window_increases(in1), 1429)

True

# [Day 2](https://adventofcode.com/2021/day/2): Dive! 

The input is a list of instructions, such as "`forward 10`".

1. Follow instructions and report the product of your final horizontal position and depth.
1. Follow *revised* instructions and report the product of your final horizontal position and depth. (There is an "aim" which is increased by down and up instructions. Depth is changed not by down and up, but by going forward *n* units, which changes depth by aim × *n* units

In [7]:
in2 = parse(2, atoms)

In [8]:
def drive(instructions) -> int:
    """What is the product of position and depth after following instructions?"""
    pos = depth = 0
    for (op, n) in instructions:
        if op == 'forward': pos += n
        if op == 'down':    depth += n
        if op == 'up':      depth -= n
    return pos * depth

answer(2.1, drive(in2), 1670340)

True

In [9]:
def drive2(instructions) -> int:
    """What is rthe product of position and depth after following instructions?
    This time we have to keep track of `aim` as well."""
    pos = depth = aim = 0
    for (op, n) in instructions:
        if op == 'forward': pos += n; depth += aim * n
        if op == 'down':    aim += n
        if op == 'up':      aim -= n
    return pos * depth

answer(2.2, drive2(in2), 1954293920)

True

# [Day 3](https://adventofcode.com/2021/day/3): Binary Diagnostic

The input is a list of bit strings.

1. What is the power consumption (product of gamma and epsilon rates)?
2. What is the life support rating (product of oxygen and CO2)?

In [10]:
in3 = parse(3, str) # Parse into bit strings, (e.g. '1110'), not binary ints (e.g. 14)

In [31]:
def common(strs, i) -> str: 
    """The bit that is most common in position i."""
    bits = [s[i] for s in strs]
    return '1' if bits.count('1') >= bits.count('0') else '0'

def uncommon(strs, i) -> str: return '1' if common(strs, i) == '0' else '0'

def epsilon(strs) -> str:
    """The bit string formed from most common bit at each position."""
    return cat(common(strs, i) for i in range(len(strs[0])))

def gamma(strs) -> str:
    """The bit string formed from most uncommon bit at each position."""
    return cat(uncommon(strs, i) for i in range(len(strs[0])))

def power(strs) -> int: 
    return int(epsilon(strs), 2) * int(gamma(strs), 2)
    
answer(3.1, power(in3), 2261546)

True

In [30]:
def select(strs, common_fn, i=0) -> str:
    """Select a str from strs according to common_fn."""
    if len(strs) == 1:
        return strs[0]
    else:
        bit = common_fn(strs, i)
        selected = [s for s in strs if s[i] == bit]
        return select(selected, common_fn, i + 1)

def life(strs) -> int: 
    return int(select(strs, common), 2) * int(select(strs, uncommon), 2)
    
answer(3.2, life(in3), 6775520)

True

# [Day 4](https://adventofcode.com/2021/day/4): Giant Squid

The first line of the input is a permutation of the integers 0-99. That is followed by 5 × 5 bingo boards, each separated by two newlines.

1. What will your final score be if you choose the first bingo board to win?
2. What will your final score be if you choose the last bingo board to win?

I'll represent a board as a tuple of 25 ints; that makes `parse` easy: the permutation of integers and the bingo boards can both be parsed by `ints`. 

I'm worried about an ambiguity: what if two boards win at the same time? I'll have to assume Eric arranged it so that can't happen. I'll define `bingo_winners` to return a list of boards that win when a number has just been called, and I'll arbitrarily choose the first of them.

In [13]:
order, *boards = in4 = parse(4, ints, sep='\n\n')

In [14]:
Board = Tuple[int]
Line = List[int]
B = 5
def sq(x, y) -> int: return x + B * y

def lines(square) -> Tuple[Line, Line]:
    """The two lines through square number `square`."""
    x, y = square % B, square // B
    return [sq(x, y) for x in range(B)], [sq(x, y) for y in range(B)]

def bingo_winners(boards, drawn, just_called) -> List[Board]:
    """Are any boards winners due to the number just called (and previously drawn numbers)?"""
    return [board for board in boards
            if just_called in board
            and any(quantify(board[n] in drawn for n in line) == B 
                    for line in lines(board.index(just_called)))]

def bingo_score(board, drawn, just_called) -> int:
    """Sum of unmarked numbers multiplied by the number just called."""
    return sum(n for n in board if n not in drawn) * just_called

def bingo(boards, order) -> int: 
    """What is the score of the first winning board?"""
    drawn = set()
    for num in order:
        drawn.add(num)
        winners = bingo_winners(boards, drawn, num)
        if winners:
            return bingo_score(winners[0], drawn, num)

answer(4.1, bingo(boards, order), 39902)

True

In [15]:
def bingo_last(boards, order) -> int: 
    """What is the score of the last winning board?"""
    boards = set(boards)
    drawn = set()
    for num in order:
        drawn.add(num)
        winners = bingo_winners(boards, drawn, num)
        boards -= set(winners)
        if not boards:
            return bingo_score(winners[-1], drawn, num)
                
answer(4.2, bingo_last(boards, order), 26936)

True

# [Day 5](https://adventofcode.com/2021/day/5): Hydrothermal Venture

The input is a list of "lines" denoted by start and end points, e.g. "`0,9 -> 5,9`". I'll represent that line as the tuple `(0, 9, 5, 9)`.

1. Consider only horizontal and vertical lines. At how many points do at least two lines overlap?
2. Consider all of the lines (including diagonals). At how many points do at least two lines overlap?

In [16]:
in5 = parse(5, ints)

In [17]:
def points(line) -> bool:
    """All the (integer) points on a line."""
    x1, y1, x2, y2 = line
    if x1 == x2:
        return [(x1, y) for y in cover(y1, y2)]
    elif y1 == y2:
        return [(x, y1) for x in cover(x1, x2)]
    else: # non-orthogonal lines not allowed
        return []
    
def cover(x1, x2) -> range:
    """All the ints from x1 to x2, inclusive, with x1, x2 in either order."""
    return range(min(x1, x2), max(x1, x2) + 1)

def overlaps(lines) -> int:
    """How many points overlap 2 or more lines?"""
    counts = Counter(flatten(map(points, lines)))
    return quantify(counts[p] >= 2 for p in counts)

answer(5.1, overlaps(in5), 7436)

True

In [18]:
def overlaps(lines, diagonal=False) -> int:
    """How many points overlap 2 or more lines?"""
    counts = Counter(flatten(points(line, diagonal) for line in lines))
    return quantify(counts[p] >= 2 for p in counts)

def points(line, diagonal=False) -> bool:
    """All the (integer) points on a line; optionally allow diagonal lines."""
    x1, y1, x2, y2 = line
    if diagonal or x1 == x2 or y1 == y2:
        dx, dy = sign(x2 - x1), sign(y2 - y1)
        length = max(abs(x2 - x1), abs(y2 - y1))
        return [(x1 + k * dx, y1 + k * dy) for k in range(length + 1)]
    else: # non-orthogonal lines not allowed when diagonal is False
        return []

assert points((1, 1, 1, 3), False) == [(1, 1), (1, 2), (1, 3)]
assert points((1, 1, 3, 3), True) == [(1, 1), (2, 2), (3, 3)]
assert points((9, 7, 7, 9), True) == [(9, 7), (8, 8), (7, 9)]

answer(5.2, overlaps(in5, True), 21104)

True

# [Day 6](https://adventofcode.com/2021/day/6): Lanternfish

The input is a single line of ints, each describing the age of a lanternfish. Over time, they age and reproduce in a specified way.

1. Find a way to simulate lanternfish. How many lanternfish would there be after 80 days?
2. How many lanternfish would there be after 256 days?

In [19]:
in6 = parse(6, ints)[0]

Although the puzzle description treats each fish individually, I won't take the bait (pun intended). Instead, I'll use a Counter of fish, and treat all the fish of each age group together. I have a hunch that part 2 will involve a ton-o'-fish.

In [25]:
Fish = Counter # Represent a school of fish as a Counter of their timers

def simulate(fish, days=1) -> Tuple[Fish, int]:
    """Simulate the aging and birth of fish over `days`;
    return the Counter of fish and the total number of fish."""
    for day in range(days):
        fish = Fish({t - 1: fish[t] for t in fish})
        if -1 in fish: # births
            fish[6] += fish[-1]
            fish[8] = fish[-1]
            del fish[-1]
    return fish, sum(fish.values())
        
assert simulate(Fish((3, 4, 3, 1, 2))) == (Fish((2, 3, 2, 0, 1)), 5)
assert simulate(Fish((2, 3, 2, 0, 1))) == (Fish((1, 2, 1, 6, 0, 8)), 6)

answer(6.1, simulate(Fish(in6), 80)[1], 350917)

True

My hunch was right, so part 2 is easy:

In [26]:
answer(6.2, simulate(Fish(in6), 256)[1], 1592918715629)

True

# [Day 7](https://adventofcode.com/2021/day/7):